In [3]:
# Set me to the location of this directory...
%cd /Users/glenngalvizo/Documents/Graduate/cs-271a/cs271-sokoban/

/Users/glenngalvizo/Documents/Graduate/cs-271a/cs271-sokoban


In [4]:
import mctsnode as nd
import numpy as np
import singplayermcts as mc
import game_model as game

rootState = game.createGame("resources/sokoban03.txt")

root = nd.Node(rootState)
x = mc.MCTS(root)

x.run(iters=100)
x.display_moves()


[2020-12-11 14:39:53,920][INFO][game_model -- createGame]: Creating game for file: resources/sokoban03.txt.
[2020-12-11 14:39:53,924][DEBUG][game_model -- __init__]: Creating new game state: {"Boxes": [(3, 3), (3, 4)], "currentLocation": (5, 2)}.
[2020-12-11 14:39:53,928][DEBUG][game_model -- createGame]: New game state: 
######
# .. #
# $$ #
#    #
#@   #
######

[2020-12-11 14:39:53,929][DEBUG][singplayermcts -- run]: ==== Starting iteration 0. ==== 
[2020-12-11 14:39:53,933][DEBUG][game_model -- legal_moves]: Current legal moves: [<Move.UP: 1>, <Move.RIGHT: 2>]
[2020-12-11 14:39:53,946][DEBUG][game_model -- __init__]: Creating new game state: {"Boxes": [(3, 3), (3, 4)], "currentLocation": (5, 2)}.
[2020-12-11 14:39:53,948][DEBUG][game_model -- makeMove]: New game state: 
######
# .. #
# $$ #
#    #
# @  #
######

[2020-12-11 14:39:53,949][DEBUG][singplayermcts -- simulation]: Current state: (5, 3)
[2020-12-11 14:39:53,951][DEBUG][game_model -- legal_moves]: Current legal moves: [<Mo

Traceback (most recent call last):
  File "/Users/glenngalvizo/Documents/Graduate/cs-271a/cs271-sokoban/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-a34e6004f190>", line 11, in <module>
    x.run()
  File "/Users/glenngalvizo/Documents/Graduate/cs-271a/cs271-sokoban/singplayermcts.py", line 150, in run
    Result = self.simulation(Y, 20)
  File "/Users/glenngalvizo/Documents/Graduate/cs-271a/cs271-sokoban/singplayermcts.py", line 96, in simulation
    currState = game.pickpossState(currState)
  File "/Users/glenngalvizo/Documents/Graduate/cs-271a/cs271-sokoban/game_model.py", line 234, in pickpossState
    A = legal_moves(currState)
  File "/Users/glenngalvizo/Documents/Graduate/cs-271a/cs271-sokoban/game_model.py", line 149, in legal_moves
    logger.debug(f'Current legal moves: {moves}')
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versi

TypeError: object of type 'NoneType' has no len()